## Code from Microsoft Site 
#### https://docs.microsoft.com/en-us/microsoft-365/enterprise/microsoft-365-ip-web-service?view=o365-worldwide
#### IP Addresses are Sorted Correctly

In [ ]:
import json
from pathlib import Path
import urllib.request
import uuid
import ipaddress


In [ ]:
# helper to call the webservice and parse the response
def webApiGet(methodName, instanceName, clientRequestId):
    ws = "https://endpoints.office.com"
    requestPath = f"{ws}/{methodName}/{instanceName}?clientRequestId={clientRequestId}"
    request = urllib.request.Request(requestPath)
    with urllib.request.urlopen(request) as response:
        return json.loads(response.read().decode())


In [ ]:
datapath = Path('endpoints_clientid_latestversion.txt')
# fetch client ID and version if data exists; otherwise create new file
if datapath.exists():
    with open(datapath, 'r') as fin:
        clientRequestId = fin.readline().strip()
        latestVersion = fin.readline().strip()
else:
    clientRequestId = str(uuid.uuid4())
    latestVersion = '0000000000'
    with open(datapath, 'w') as fout:
        fout.write(clientRequestId + '\n' + latestVersion)


In [ ]:
# call version method to check the latest version, and pull new data if version number is different

version = webApiGet('version', 'Worldwide', clientRequestId)

if version['latest'] > latestVersion:
    print('\nNew version of Office 365 worldwide commercial service instance endpoints detected')

    # write the new version number to the data file
    with open(datapath, 'w') as fout:
        fout.write(clientRequestId + '\n' + version['latest'])

    # invoke endpoints method to get the new data
    endpointSets = webApiGet('endpoints', 'Worldwide', clientRequestId)

    flatIps = []
    for endpointSet in endpointSets:
        if endpointSet['category'] in ('Optimize', 'Allow'):
            ips = endpointSet['ips'] if 'ips' in endpointSet else []
            ip4s = [ip for ip in ips if '.' in ip]
            flatIps.extend([ip for ip in ip4s])

    print('\nIPv4 Firewall IP Address Ranges')
    print(','.join(sorted(set(flatIps), key=ipaddress.IPv4Network)))                        # IP Address Sort

    with open('IPv4 Firewall IP Address Ranges.txt', 'w') as output_file:
        output_file.write('\n'.join(sorted(set(flatIps), key=ipaddress.IPv4Network)))       # IP Address Seort

    flatUrls = []
    for endpointSet in endpointSets:
        if endpointSet['category'] in ('Optimize', 'Allow'):
            urls = endpointSet['urls'] if 'urls' in endpointSet else []
            flatUrls.extend([url for url in urls])

    print('\nURLs for Proxy Server')
    print(','.join(sorted(set(flatUrls))))

    with open('URLs for Proxy Server.txt', 'w') as output_file:
        output_file.write('\n'.join(sorted(set(flatUrls))))

else:
    print('Office 365 worldwide commercial service instance endpoints are up-to-date')
